In [ ]:
import cv2
import numpy as np

## function to draw ROI and distance for threshold calculation 

In [ ]:
def get_mouse_points(event, x, y, flags, param):

    global mouseX, mouseY, mouse_pts
    if event == cv2.EVENT_LBUTTONDOWN:
        mouseX, mouseY = x, y
        cv2.circle(image, (x, y), 10, (0, 255, 255), 10)
        if "mouse_pts" not in globals():
            mouse_pts = []
        mouse_pts.append((x, y))
        print("Point detected")
        print(mouse_pts)

In [ ]:
def get_mouse_points_ped(event, x, y, flags, param):

    global mouseX, mouseY, mouse_pts_ped
    if event == cv2.EVENT_LBUTTONDOWN:
        mouseX, mouseY = x, y
        cv2.circle(img_ped, (x, y), 10, (0, 255, 0), 10)
        if "mouse_pts_ped" not in globals():
            mouse_pts_ped = []
        mouse_pts_ped.append((x, y))
        print("Point detected")
        print(mouse_pts_ped)

## Drawing points

In [ ]:
img = cv2.imread("images/baseImg.PNG")
height = int(cv2.CAP_PROP_FRAME_HEIGHT)
width = int(cv2.CAP_PROP_FRAME_WIDTH)

In [ ]:
flag = 1

def test():
    
    def get_mouse_points(event, x, y, flags, param):

        global mouseX, mouseY, mouse_pts
        if event == cv2.EVENT_LBUTTONDOWN:
            mouseX, mouseY = x, y
            cv2.circle(image, (x, y), 10, (0, 255, 255), 10)
            if "mouse_pts" not in globals():
                mouse_pts = []
            mouse_pts.append((x, y))
            print("Point detected")
            print(mouse_pts)
    
    mouse_pts = []
    cv2.namedWindow("image")
    cv2.setMouseCallback("image", get_mouse_points)
    num_mouse_points = 0
    while True:
        image = img
        cv2.imshow("image", image)
        cv2.waitKey(1)
        if len(mouse_pts) == 6:
            cv2.destroyWindow("image")
        break
    four_points = mouse_pts
    
    return four_points

In [ ]:
test()

In [ ]:
img_ped = cv2.imread("images/baseImg.PNG")

In [ ]:
mouse_pts_ped = []
cv2.namedWindow("ped")
cv2.setMouseCallback("ped", get_mouse_points_ped)
num_mouse_points_ped = 0
while True:
    cv2.imshow("ped", img_ped)
    cv2.waitKey(1)
    if len(mouse_pts_ped) == 5:
        cv2.destroyWindow("ped")
        break
ped_box = mouse_pts_ped

In [ ]:
def get_camera_perspective(img, src_points):
    IMAGE_H = img.shape[0]
    IMAGE_W = img.shape[1]
    src = np.float32(np.array(src_points))
    dst = np.float32([[0, IMAGE_H], [IMAGE_W, IMAGE_H], [0, 0], [IMAGE_W, 0]])

    M = cv2.getPerspectiveTransform(src, dst)
    M_inv = cv2.getPerspectiveTransform(dst, src)

    return M, M_inv

In [ ]:
frame = img
M, Minv = get_camera_perspective(frame, four_points[0:4])

frame_h = frame.shape[0]
frame_w = frame.shape[1]

scale_w = 1.2 / 2
scale_h = 4 / 2

In [ ]:
pts = src = np.float32(np.array([four_points[4:]]))
warped_pt = cv2.perspectiveTransform(pts, M)[0]
d_thresh = np.sqrt((warped_pt[0][0] - warped_pt[1][0]) ** 2 + (warped_pt[0][1] - warped_pt[1][1]) ** 2)
bird_image = np.zeros((int(frame_h * scale_h), int(frame_w * scale_w), 3), np.uint8)

bird_image[:] = (0, 0, 0) #black image

In [ ]:
# draw polygon of ROI
pts = np.array([four_points[0], four_points[1], four_points[3], four_points[2]], np.int32)
cv2.polylines(frame, [pts], True, (0, 255, 255), thickness=4)

In [ ]:
def plot_points_on_bird_eye_view(frame, pedestrian_boxes, M, scale_w, scale_h):
    frame_h = frame.shape[0]
    frame_w = frame.shape[1]

    node_radius = 10
    color_node = (192, 133, 156)
    thickness_node = 20
    solid_back_color = (41, 41, 41)

    blank_image = np.zeros(
        (int(frame_h * scale_h), int(frame_w * scale_w), 3), np.uint8
    )
    blank_image[:] = (41, 41, 41)
    warped_pts = []
    for i in range(len(pedestrian_boxes)):
        """
        mid_point_x = int(
            (pedestrian_boxes[i][1] * frame_w + pedestrian_boxes[i][3] * frame_w) / 2
        )
        mid_point_y = int(
            (pedestrian_boxes[i][0] * frame_h + pedestrian_boxes[i][2] * frame_h) / 2
        )
        """
        mid_point_x = int(pedestrian_boxes[i][0] * frame_w)
        mid_point_y = int(pedestrian_boxes[i][1] * frame_h)

        pts = np.array([[[mid_point_x, mid_point_y]]], dtype="float32")
        warped_pt = cv2.perspectiveTransform(pts, M)[0][0]
        warped_pt_scaled = [int(warped_pt[0] * scale_w), int(warped_pt[1] * scale_h)]

        warped_pts.append(warped_pt_scaled)
        bird_image = cv2.circle(
            blank_image,
            (warped_pt_scaled[0], warped_pt_scaled[1]),
            node_radius,
            color_node,
            thickness_node,
        )

    return warped_pts, bird_image

In [ ]:
frame_img = cv2.imread("images/baseImg.PNG")

In [ ]:
w_pts, bird = plot_points_on_bird_eye_view(frame_img, ped_box, M, scale_w, scale_h)

In [ ]:
cv2.imshow("bird", bird)
cv2.waitKey(5000)
cv2.destroyWindow("bird")

In [ ]:
for i in range(0, len(ped_box)):
    print(i, " : ")
    print("x = ", ped_box[i][0], ", y = ", ped_box[i][1])

In [ ]:
w_pts

In [ ]:
frame_h = frame_img.shape[0]
frame_w = frame_img.shape[1]

node_radius = 30
color_node = (192, 133, 156)
thickness_node = 20
solid_back_color = (41, 41, 41)

blank_image = np.zeros((int(frame_h * scale_h), int(frame_w * scale_w), 3), np.uint8)
blank_image[:] = (0, 0, 0)

for i in range(len(w_pts)):
    bird_image = cv2.circle(
                blank_image,
                (w_pts[i][0], w_pts[i][1]),
                node_radius,
                color_node,
                thickness_node,
            )


In [ ]:
cv2.imshow("bird_test", bird_image)
cv2.waitKey(100000)
cv2.destroyWindow("bird_test")

In [ ]:
result = cv2.warpPerspective(frame_img, M, (int(frame_h * scale_h), int(frame_w * scale_w)))

In [ ]:
cv2.imshow("warp", result)
cv2.waitKey(10000)
cv2.destroyWindow("warp")

In [ ]:
node_radius = int(d_thresh)
color_node = (192, 133, 156)
thickness_node = 10
solid_back_color = (41, 41, 41)

for i in range(len(w_pts)):
    r_img = cv2.circle(
                result,
                (w_pts[i][0], w_pts[i][1]),
                node_radius,
                color_node,
                thickness_node,
            )

In [ ]:
cv2.imshow("R_warp", r_img)
cv2.waitKey(10000)
cv2.destroyWindow("R_warp")

In [ ]:
f_img = cv2.circle(
                result,
                (100, 100),
                node_radius,
                color_node,
                thickness_node,
            )

In [ ]:
cv2.imshow("R_warp", f_img)
cv2.waitKey(10000)
cv2.destroyWindow("R_warp")